In [22]:
"""
CELL 1: SETUP (WITH TIMING)
Description: Load configuration and all VAANI functions with timing
"""

import time

print("="*60)
print("🎮 VAANI PLAYGROUND - LOADING")
print("="*60)

# Time Config Loading
config_start = time.time()
%run vyapar_config.ipynb
config_time = time.time() - config_start
print(f"✅ Config loaded: {config_time*1000:.0f} ms")

# Time Prompts Loading
prompts_start = time.time()
%run vaani_prompts.ipynb
prompts_time = time.time() - prompts_start
print(f"✅ Prompts loaded: {prompts_time*1000:.0f} ms")

# Time Functions Loading
functions_start = time.time()
%run vaani_functions.ipynb
functions_time = time.time() - functions_start
print(f"✅ Functions loaded: {functions_time*1000:.0f} ms")

# Summary
total_time = config_time + prompts_time + functions_time
print("\n" + "="*60)
print(f"⏱️ TOTAL LOAD TIME: {total_time*1000:.0f} ms ({total_time:.2f}s)")
print("="*60)
print(f"   Config:    {config_time*1000:>6.0f} ms ({config_time/total_time*100:>4.1f}%)")
print(f"   Prompts:   {prompts_time*1000:>6.0f} ms ({prompts_time/total_time*100:>4.1f}%)")
print(f"   Functions: {functions_time*1000:>6.0f} ms ({functions_time/total_time*100:>4.1f}%)")
print("="*60)
print("✅ Ready to test!")

🎮 VAANI PLAYGROUND - LOADING
✅ Libraries imported successfully
✅ Master Registry Link configured
📍 Link: https://docs.google.com/spreadsheets/d/e/2PACX-1vQdOVYDNLuMG...

🔄 Testing registry load...
✅ Registry loaded successfully: 9 items found

📋 Available items in registry:
  1. default_model
  2. openai_api_key
  3. anthropic_api_key
  4. master_registry_link
  5. usage_data
  6. dropoff_analysis
  7. user_research
  8. excel_db_1000_items
  9. claude_haiku

🧪 Testing get_item() function:

1. Testing with 'default_model':
✅ Registry loaded successfully: 9 items found
✅ Retrieved 'default_model'
   Result: {'model': 'claude-sonnet-4-20250514', 'temperature': 0.3, 'max_tokens': 1000, 'provider': 'anthropic'}

2. Testing with non-existent item:
✅ Registry loaded successfully: 9 items found
❌ Item 'this_does_not_exist' not found in registry
💡 Available items: default_model, openai_api_key, anthropic_api_key, master_registry_link, usage_data, dropoff_analysis, user_research, excel_db_1000_

In [18]:
"""
CELL 2: QUICK SINGLE TEST
"""

timer = CellTimer()

# ============ MODIFY THESE ============
test_input = "chai samosa 140 rupees"
transaction_type = None  # Optional: 'expense', 'sale', etc.
# ======================================

print("="*60)
print(f"📝 Testing: '{test_input}'")
print("="*60)

result = route_with_intent(test_input, transaction_type=transaction_type)

print(f"\n📊 RESULT:")
print(f"Status: {result.get('status')}")

if result.get('status') == 'success':
    print(f"Transaction Type: {result.get('transaction_type')}")
    
    print(f"\n✅ Extracted Data:")
    extraction = result.get('extraction', {})
    for key, value in extraction.items():
        if key not in ['raw_response', 'transaction_type']:
            print(f"   {key}: {value}")
    
    if 'time_taken' in extraction:
        print(f"\n⏱️ API Time: {extraction['time_taken']*1000:.0f} ms")

elif result.get('status') == 'not_relevant':
    print(f"⚠️ {result.get('message')}")
else:
    print(f"❌ Error: {result.get('error')}")

timer.print_summary("TOTAL")

📝 Testing: 'chai samosa 140 rupees'
🔍 Running intent detection...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized

📊 RESULT:
Status: success
Transaction Type: expense

✅ Extracted Data:
   amount: 140
   item: chai and samosa
   category: Food & Beverages
   model_used: claude-sonnet-4-20250514
   time_taken: 4.82

⏱️ API Time: 4820 ms

⏱️ TOTAL TIME: 14994 ms (14.99s)


In [19]:
"""
CELL 3: ENHANCED BATCH TEST
Description: Test multiple inputs across Sonnet and Haiku
"""

timer = CellTimer()

# ============ TEST CONFIGURATION ============
test_cases = [
    # Expenses
    {"input": "chai samosa 140 rupees", "type": "expense"},
    {"input": "petrol 500 rupees", "type": "expense"},
    {"input": "taxi ke liye 200 diye", "type": "expense"},
    {"input": "delivery charges 50", "type": "expense"},
    
    # Sales
    {"input": "Sharma ji bought 5kg rice for 250 rupees", "type": "sale"},
    {"input": "sold vegetables to Ramesh 300 rupees", "type": "sale"},
    
    # Payments In
    {"input": "Mishra aunty ne 500 ka udhar chukaya", "type": "payment_in"},
    {"input": "received 2000 from Kumar", "type": "payment_in"},
    
    # Payments Out
    {"input": "paid 1000 rupees to supplier", "type": "payment_out"},
    {"input": "Gupta ji ko 5000 diye", "type": "payment_out"},
    
    # Not Relevant
    {"input": "what's the weather today?", "type": None},
    {"input": "hello how are you", "type": None},
]

models = [
    ('claude-sonnet-4-5-20250929', 'Sonnet'),
    ('claude-haiku-4-5-20251001', 'Haiku'),
]

skip_intent_modes = [
    (False, 'Auto-detect'),
    (True, 'Skip Intent'),
]
# ==========================================

print("="*60)
print(f"🧪 COMPREHENSIVE BATCH TEST")
print(f"Test Cases: {len(test_cases)}")
print(f"Models: {len(models)}")
print(f"Modes: {len(skip_intent_modes)}")
print(f"Total Tests: {len(test_cases) * len(models) * len(skip_intent_modes)}")
print("="*60)

all_results = []

for case_idx, test_case in enumerate(test_cases, 1):
    test_input = test_case['input']
    expected_type = test_case['type']
    
    print(f"\n[{case_idx}/{len(test_cases)}] '{test_input[:45]}...'")
    
    for model_id, model_name in models:
        for skip_intent, mode_name in skip_intent_modes:
            
            if skip_intent and expected_type:
                tx_type_param = expected_type
            else:
                tx_type_param = None
            
            result = route_with_intent(test_input, transaction_type=tx_type_param)
            
            row = {
                'input': test_input[:40],
                'expected_type': expected_type or 'not_relevant',
                'model': model_name,
                'mode': mode_name,
                'status': result.get('status'),
                'detected_type': result.get('transaction_type', 'N/A'),
                'time_ms': 'N/A',
                'amount': 'N/A',
            }
            
            if result.get('status') == 'success':
                extraction = result.get('extraction', {})
                row['time_ms'] = int(extraction.get('time_taken', 0) * 1000)
                row['amount'] = extraction.get('amount', 'N/A')
                status_icon = '✅'
            elif result.get('status') == 'not_relevant':
                status_icon = '⚠️'
            else:
                status_icon = '❌'
                row['amount'] = 'ERROR'
            
            all_results.append(row)
            print(f"   {status_icon} {model_name:6} | {mode_name:12} | {row['time_ms']:>6} ms")

df = pd.DataFrame(all_results)

# Summary by Model
print("\n" + "="*60)
print("📊 SUMMARY BY MODEL")
print("="*60)

for model_name in ['Sonnet', 'Haiku']:
    model_df = df[df['model'] == model_name]
    success_count = len(model_df[model_df['status'] == 'success'])
    
    print(f"\n🔹 {model_name}:")
    print(f"   Success: {success_count}/{len(model_df)} ({success_count/len(model_df)*100:.1f}%)")
    
    times = pd.to_numeric(model_df[model_df['time_ms'] != 'N/A']['time_ms'], errors='coerce')
    if len(times) > 0:
        print(f"   Avg Time: {times.mean():.0f} ms")
        print(f"   Min Time: {times.min():.0f} ms")
        print(f"   Max Time: {times.max():.0f} ms")

# Summary by Mode
print("\n" + "="*60)
print("📊 SUMMARY BY MODE")
print("="*60)

for mode_name in ['Auto-detect', 'Skip Intent']:
    mode_df = df[df['mode'] == mode_name]
    success_count = len(mode_df[mode_df['status'] == 'success'])
    
    print(f"\n🔹 {mode_name}:")
    print(f"   Success: {success_count}/{len(mode_df)} ({success_count/len(mode_df)*100:.1f}%)")
    
    times = pd.to_numeric(mode_df[mode_df['time_ms'] != 'N/A']['time_ms'], errors='coerce')
    if len(times) > 0:
        print(f"   Avg Time: {times.mean():.0f} ms")

# Overall accuracy
print("\n" + "="*60)
print("📈 OVERALL ACCURACY")
print("="*60)

correct = len(df[(df['expected_type'] == df['detected_type']) | 
                  ((df['expected_type'] == 'not_relevant') & (df['status'] == 'not_relevant'))])
total = len(df)
print(f"Correct Type Detection: {correct}/{total} ({correct/total*100:.1f}%)")

timer.print_summary("TOTAL BATCH TEST")
print(f"⏱️ Avg per test: {timer.elapsed()/len(all_results)*1000:.0f} ms")
print("="*60)

🧪 COMPREHENSIVE BATCH TEST
Test Cases: 12
Models: 2
Modes: 2
Total Tests: 48

[1/12] 'chai samosa 140 rupees...'
🔍 Running intent detection...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
   ✅ Sonnet | Auto-detect  |   6370 ms
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
   ✅ Sonnet | Skip Intent  |   5740 ms
🔍 Running intent detection...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
 

In [20]:
"""
CELL 4: VIEW PROMPTS & FIELDS
"""

# ============ MODIFY THIS ============
what_to_view = 'fields'  # Options: 'fields', 'prompts', 'expense', 'sale', 'intent', etc.
# =====================================

print("\n")

if what_to_view == 'fields':
    view_transaction_fields()
elif what_to_view == 'prompts':
    view_all_prompts()
else:
    view_prompt(what_to_view)



📊 TRANSACTION FIELD REQUIREMENTS

🔹 EXPENSE
   Description: Recording business expenses
   ✅ Necessary: amount, item
   📝 Additional: category, date, payment_type, notes

🔹 SALE
   Description: Recording sales/invoices
   ✅ Necessary: customer_name, amount, items
   📝 Additional: payment_type, date, notes, invoice_number

🔹 PURCHASE
   Description: Recording purchases from suppliers
   ✅ Necessary: supplier_name, amount, items
   📝 Additional: payment_type, date, notes, inventory_update

🔹 PAYMENT_IN
   Description: Money received from customers
   ✅ Necessary: payer_name, amount
   📝 Additional: payment_type, date, notes, invoice_reference

🔹 PAYMENT_OUT
   Description: Money paid to vendors/suppliers
   ✅ Necessary: payee_name, amount
   📝 Additional: payment_type, date, notes, invoice_reference



In [21]:
"""
CELL 5: CUSTOM TEST AREA
Description: Free space for experiments
"""

timer = CellTimer()

# Your custom code here
# Example:
# result = route_with_intent("your test input")
# print(result)

print("💡 Use this cell for custom experiments")

timer.print_summary("CUSTOM TEST")

💡 Use this cell for custom experiments

⏱️ CUSTOM TEST TIME: 0 ms (0.00s)
